In [ ]:
!mkdir facades
!unzip archive\ \(2\).zip -d facades

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

from i2i.datasets.facades import FacadesDataset
from i2i.datasets.collator import Collator
from i2i.model.unet import UNet
from i2i.trainer.simple_trainer import train_epoch
from i2i.logger.wandb import WanDBWriter
from i2i.utils.utils import count_parameters
from i2i.loss.reconstruction import ReconstructionLoss

In [3]:
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd
from torch.utils.data import DataLoader

from itertools import repeat

In [4]:
config = dict(
    output_ch_size = 3,
    shape = (256,256),
    input_ch_size = 3,
    hidden_ch_size = 16,
    feature_levels_num = 3,
    block_depth = 3,
    batch_size = 16,
    n_epochs = 25,
    lr = 4e-3,
    overfit_batch = True,
    wandb_project = 'Image2Image',
    device = 'cpu'
)

In [9]:
dataset = FacadesDataset('facades/')

collator = Collator()
train_loader = DataLoader(dataset, batch_size=config['batch_size'], collate_fn=collator)

model = UNet(**config)

opt = torch.optim.AdamW(model.parameters(), lr=config['lr'])

loss = ReconstructionLoss()

count_parameters(model)

1210957

In [10]:
logger = WanDBWriter(config)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


train/l2_loss,▁
train/steps_per_sec,▁
train/l2_loss,0.5504
train/steps_per_sec,4.58008


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [11]:
if config['overfit_batch']:
    train_loader = iter(train_loader)
    batch = next(train_loader)
    train_loader = repeat(batch)

In [ ]:
train_epoch(model, opt, train_loader, loss, config, logger=logger)

0it [00:00, ?it/s]